# Lab 3 - PNA Parser Implementation
<div style="text-align: justify;">
This lab will walk you through creating an experiment that contains a P4-DPDK programmable pipeline. This lab starts by describing how to define custom headers in a P4 program. It then explains how to implement a simple parser that parses the defined headers. The lab further shows how to track the parsing states of a packet in the DPDK pipeline.
<figure style="text-align: center;">
    <img src="./labs_files/lab3/figs/0_00_fabric_topology.png" width="550px"><br>

# Introduction

## Program headers and definitions

<div style="text-align: justify;">
For several decades, the networking industry operated in a bottom-up approach. At the bottom of the system are the fixed-function Application Specific Integrated Circuits (ASICs) or other domain-specific hardware, which enforce protocols, features, and processes available in the switch or the Network Interface Card (NIC). Programmers and operators are limited to these capabilities when building their systems. Consequently, systems have features defined by the vendors that are rigid and may not fit the network operators’ needs. P4 allows agility in the networking industry by enabling a top-down approach to the design of network applications. With this approach, the programmer or network operator can precisely describe features and how packets are processed in the packet processing pipeline. </div>   <br> 

<div style="text-align: justify;">
With the Portable NIC Architecture (PNA) [<a href="#References">1</a>], the programmer defines the headers and corresponding parser as well as actions executed in the pipeline and the deparser. The programmer has the flexibility of defining custom headers (i.e., a header not standardized). Such capability is not available in non-programmable devices. The Data Plane Development Kit (DPDK) accelerates packet processing and enhances performance. With P4-DPDK, the programmability features of P4 and the acceleration capabilities of DPDK come together. </div>   <br> 

<figure style="text-align: center;">
  <img src="./labs_files/lab3/figs/0_intro_1_ethernet.png" width="400" style="display: block; margin: 0 auto;">
  <figcaption>Figure 1. Ethernet header.</figcaption>
</figure>
<figure style="text-align: center;">
  <img src="./labs_files/lab3/figs/0_intro_2_IP.png" width="750" style="display: block; margin: 0 auto;">
  <figcaption>Figure 2. Ethernet header.</figcaption>
</figure>
<figure style="text-align: center;">
  <img src="./labs_files/lab3/figs/0_intro_3_TCP.png" width="750" style="display: block; margin: 0 auto;">
  <figcaption>Figure 3. Ethernet header.</figcaption>
</figure>

<div style="text-align: justify;">
Figure 4 shows an excerpt of a P4 program where a header is defined. This is typically written at the top of the program before the parsing starts. We can see that the programmer-defined header corresponds to Ethernet (lines 11-14). The Ethernet header fields are shown in Figure 1. The programmer also defined an IPv4 header (lines 16-29) and a TCP header (lines 31-42). The IPv4 header format is shown in Figure 2 and the TCP header is shown in Figure 3.
 </div>   <br> 

 <figure style="text-align: center;">
  <img src="./labs_files/lab3/figs/0_intro_4_P4_headers.png" width="550" style="display: block; margin: 0 auto;">
  <figcaption>Figure 4. Ethernet header.</figcaption>
</figure>

The code starts by including the ```core.p4``` file (line 1) which defines some common types and variables used in all P4 programs. For instance, the ```packet_in``` and ```packet_out``` extern types which represent incoming and outgoing packets, respectively, are declared in ```core.p4``` [<a href="#References">2</a>]. Next, the ```pna.p4``` [<a href="#References">3</a>] file is included (line 2) to define the PNA architecture and all its externs used when writing P4 programs [<a href="#References">4</a>]. Line 3 defines a 16-bit constant ```TYPE_IPV4``` with the value ```0x800```. Similarly, line 4 creates an 8-bit constant ```TYPE_TCP``` with the value ```6```. This means that these variables can be used later in the P4 program to reference the associated values. The typedef declarations (lines 8-9) are used to assign alternative names to types. 

The program in Figure 4 defines the Ethernet header (lines 11-14). The declarations inside the header are usually written after referring to the standard specifications of the protocol. Note that in the ```ethernet_t``` header, the ```EthernetAddress``` is used rather than using a 48-bit field. The IPv4 header is also defined (lines 16-29) following the standard header specification shown in Figure 2. Note that in the ```ipv4_t``` header, the ```IP4Address``` is used rather than using a 32-bit field. Similarly, the TCP header is defined (lines 31-42) following the standard header specification shown in Figure 3. Lines 44 - 45 show how to declare user-defined metadata, which is passed from one block to another as the packet propagates through the architecture. For simplicity, this program does not require any user metadata. Finally, the headers struct (lines 47-50) that will be used in the program are defined. The headers are customized depending on how the programmer wants the packets to be parsed.


## Programmable parser

<div style="text-align: justify;">
The programmable parser permits the programmer to describe how the packets will be processed. The parser de-encapsulates the headers, converting the original packet into a parsed representation of the packet. The parser can be represented as a state machine without cycles (direct acyclic graph), with one initial state (start) and two final states (accept or reject).</div>   <br> 

 <figure style="text-align: center;">
  <img src="./labs_files/lab3/figs/0_intro_5_parser.png" width="550" style="display: block; margin: 0 auto;">
  <figcaption>(a)</figcaption>
</figure>
<figure style="text-align: center;">
  <img src="./labs_files/lab3/figs/0_intro_6_P4_parser.png" width="550" style="display: block; margin: 0 auto;">
  <figcaption>(b)</figcaption>
</figure>

Figure 5. Example of a parser. (a) Graphical representation of the parser. (b) In P4, the parser always starts with the initial state called ```start```. First, we transition unconditionally to ```parse_ethernet```. Then, we can create some conditions to direct the parser. Finally, when we transition to the ```accept``` state, the packet is moved to the control block of the pipeline. A packet that reaches the ```reject``` state will be dropped.

Figure 5a shows the graphical representation of the parser and Figure 5b its corresponding P4 code. Note that the packet is an instance of the ```packet_in``` extern  and is passed as a parameter to the parser. The ```extract``` method associated with the packet extracts N bits, where N is the total number of bits defined in the corresponding header (for example, 112 bits for Ethernet). Afterward, the ```etherType``` field of the Ethernet header is examined using the select statement, and the program branches to the ```parse_ipv4``` state if the ```etherType``` field corresponds to IPv4. The state transitions to the ```reject``` if it is not an IPv4 header, as shown in the figure above (Line 15). In the ```parse_ipv4``` state, the ```protocol``` field of the IPv4 header is examined using the select statement, and the program branches to the ```parse_tcp``` state if the ```protocol``` field corresponds to TCP. The state transitions to the ```reject``` if it is not a TCP header, as shown in the figure above (Line 23). Finally, in the ```parse_tcp``` state, the TCP header is extracted, and the program unconditionally transitions to the ```accept``` state.

# Step 1:  Configuring the environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../../configure_and_validate.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).

# Step 2: Importing the FABlib library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

# Step 3: Creating the experiment slice

The following creates a node with basic compute and networking capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

### Step 3.1: Creating a slice
The code below creates a new slice with the name "P4DPDK_lab3_vnic"

In [2]:
slice = fablib.new_slice(name="P4DPDK_lab3_vnic")

### Step 3.2: Defining the sites
The code below requests a random site from FABRIC based on the condition that the following resources are available:

<ul>
    <li> 4 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size
</ul>

In [3]:
site1= fablib.get_random_sites(count=1, filter_function=lambda x: x['cores_available'] > 4 and x['ram_available'] > 8 and x['disk_available'] > 20)[0]

print (f'The selected site is {site1}')

The selected site is UCSD


### Step 3.3: Creating the nodes
The code below creates one node (server1) which uses the following:
<ul>
    <li> 4 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

server1 will be created in site1

In [4]:
server1 = slice.add_node(name="server1", 
                      site=site1, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

### Step 3.4: Submitting the slice
The code below submits the slice. 
By default, the submit function will block until the node is ready and will display the progress of your slice being built.

In [5]:
slice.submit();


Retry: 10, Time: 226 sec


ID,588d5548-e8bc-42b5-b209-cb0a376ba4db
Name,P4DPDK_lab3_vnic
Lease Expiration (UTC),2024-09-03 20:20:13 +0000
Lease Start (UTC),2024-09-02 20:20:13 +0000
Project ID,8eaa3ec2-65e7-49a3-8c09-e1761141a6ad
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
dc9ef7a2-909c-4e5b-a337-9be791b105b6,server1,4,8,100,default_ubuntu_20,qcow2,ucsd-w2.fabric-testbed.net,UCSD,ubuntu,132.249.252.178,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.178,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


# Step 4: Installing the required packages
In this step, we will install the required packages to run the lab. Specifically, we will install the DPDK library, the P4 compiler (p4c), and all needed dependencies.

### Step 4.1 Getting the server node
The command below gets the fablib node that server1 is associated with.

In [6]:
server1 = slice.get_node(name="server1")

### Step 4.2 NAT64 setup
The code below checks if an IPv6 address is available to set up NAT64. We will upload the script [scripts/nat64.sh](./scripts/nat64.sh) to the all servers and execute it

In [7]:
from ipaddress import ip_address, IPv6Address

if type(ip_address(server1.get_management_ip())) is IPv6Address:
    server1.upload_file('scripts/nat64.sh', 'nat64.sh')
    stdout, stderr = server1.execute(f'chmod +x nat64.sh && ./nat64.sh', quiet=True)

### Step 4.3 Installing dependencies
The code below installs packages that are prerequisites to the upcoming installations and needed to run the lab experiments

In [59]:
stdout, stderr = server1.execute(f'sudo apt-get update', quiet = True)
stdout, stderr = server1.execute(f'sudo apt-get install -y build-essential python3-pip python3-pyelftools libnuma-dev pkg-config net-tools', quiet = True)
stdout, stderr = server1.execute(f'sudo pip3 install meson ninja scapy', quiet = True)

### Step 4.4 Installing DPDK
The code below downloads, builds, and installs DPDK on all servers. In this lab, we are building a modified version of DPDK in which we enabled logs in the terminal to have a better understanding of the behavior of the built pipeline. Note that printing out logs on the terminal degrades performance. Therefore, with applications where high performance is needed, logging should be disabled.  

In [9]:
stdout, stderr = server1.execute(f'git clone http://dpdk.org/git/dpdk', quiet = True)
stdout, stderr = server1.execute(f'cd dpdk/lib/pipeline/ &&  sudo rm rte_swx_pipeline.c && sudo rm rte_swx_pipeline_internal.h', quiet = True)
server1.upload_file('scripts/rte_swx_pipeline.c','/home/ubuntu/dpdk/lib/pipeline/rte_swx_pipeline.c')
server1.upload_file('scripts/rte_swx_pipeline_internal.h','/home/ubuntu/dpdk/lib/pipeline/rte_swx_pipeline_internal.h')
stdout, stderr = server1.execute(f'cd dpdk &&  sudo meson build && cd build && sudo ninja && sudo ninja install && sudo ldconfig', quiet = True)
stdout, stderr = server1.execute(f'cd dpdk/examples/pipeline && sudo make', quiet=True)

### Step 4.5 Install p4c
The code below downloads and installs the p4c compiler needed to compile the p4 code into a DPDK pipeline

In [10]:
stdout, stderr = server1.execute('git clone https://github.com/CILab-USC/p4c.git', quiet = True)
stdout, stderr = server1.execute('sudo apt-get install -y cmake g++ git automake libtool libgc-dev bison flex libfl-dev libboost-dev libboost-iostreams-dev libboost-graph-dev llvm pkg-config python3 python3-pip tcpdump', quiet = True)
stdout, stderr = server1.execute('cd p4c && pip3 install --user -r requirements.txt && mkdir build && cd build && cmake .. && make -j4  && sudo make install', quiet = True)

# Step 5: Writing and compiling the P4 code
This section demonstrates how to define custom headers in a P4 program. It also shows how to use constants and typedefs to make the program more readable.

## Step 5.1: Defining headers in the headers.p4 file

Click on [headers.p4](./labs_files/lab3/headers.p4) to open the code in the editor.

<img src="./labs_files/lab3/figs/5_01_01.png" width="700px"><br>
We can see that the headers.p4 file is empty and we have to fill it.

<hr>

We will start by defining some typedefs. Write the following in the headers.p4 file.

    typedef bit<48> EthernetAddress;
    typedef bit<32> IP4Address;
    const bit<16> TYPE_IPV4 = 0x0800;
    const bit<8> TYPE_TCP = 6;

<img src="./labs_files/lab3/figs/5_01_02.png" width="700"><br>

In the figure above the typedef declaration used (lines 2 - 5) is used to assign alternative names to types. Here we are saying that ```EthernetAddress``` can be used instead of ```bit<48>``` and ```IP4Address``` instead of ```bit<32>```.  We will use this typedef when defining the headers. Lines 4 and 5 show how to define constants with the name ```TYPE_IPV4``` set to a value of ```0x800``` and the name ```TYPE_TCP``` set to a value of ```6```. We will use these values in the parser implementation.

<hr>

Now we will define the Ethernet header. Add the following code to the headers.p4 file.

    header ethernet_t {
        EthernetAddress dstAddr;
        EthernetAddress srcAddr;
        bit<16> etherType; }

<img src="./labs_files/lab3/figs/5_01_03.png" width="700"><br>

Note how we used the typedef ```EthernetAddress``` which corresponds to ```bit<48>``` when defining the destination MAC address field (```dstAddr```) and the source MAC address field (```srcAddr```). Note how we are mapping the fields to those defined in the standard Ethernet header (see Figure 1). 

<hr>

Now we will define the IPv4 header. Add the following code to the headers.p4 file.

    header ipv4_t {
        bit<4>     version;
        bit<4>     ihl;
        bit<8>     diffserv;
        bit<16>    totalLen;
        bit<16>    identification;
        bit<3>     flags;
        bit<13>    fragOffset;
        bit<8>     ttl;
        bit<8>     protocol;
        bit<16>    hdrChecksum;
        IP4Address srcAddr;
        IP4Address dstAddr; }


<img src="./labs_files/lab3/figs/5_01_04.png" width="700"><br>

Consider the figure above. Note how we used the typedef ```IP4Address``` which corresponds to ```bit<32>``` when defining the source IP address field (```srcAddr```) and the destination IP address field (```dstAddr```). Also, note how we are mapping the fields to those defined in the standard IPv4 header (see Figure 2).

<hr>

Now we will define the TCP header. Add the following code to the headers.p4 file.

    header tcp_t {
        bit<16>  srcPort;
        bit<16>  dstPort;
        bit<32>  seqNo;
        bit<32>  ackNo;
        bit<4>   dataOffset;
        bit<3>   res;
        bit<3>   ecn;
        bit<6>   ctrl;
        bit<16>  window;
        bit<16>  checksum;
        bit<16>  urgentPtr; }


<img src="./labs_files/lab3/figs/5_01_05.png" width="700"><br>

Consider the figure above. Note how we are mapping the fields to those defined in the standard TCP header (see Figure 3). The Flags field which is usually represented in 9 bits is split into two: Explicit Congestion Notification Flags (ecn) represented by 3 bits and the Control Flags (ctrl) represented by 6 bits.

<hr>

Now we will create a struct to represent our metadata. Metadata is passed from one block to another as the packet propagates through the architecture. For simplicity, this program does not require any user metadata, and hence we will define it as empty with no fields. Add the following to the headers.p4 file.

    struct metadata {
        /* empty */
    }

<img src="./labs_files/lab3/figs/5_01_06.png" width="700"><br>

<hr>

Now we will create a struct to represent our metadata. Metadata is passed from one block to another as the packet propagates through the architecture. For simplicity, this program does not require any user metadata, and hence we will define it as empty with no fields. Add the following to the headers.p4 file.

    struct headers {
        ethernet_t   ethernet;
        ipv4_t       ipv4;
        tcp_t        tcp;}

<img src="./labs_files/lab3/figs/5_01_07.png" width="700"><br>

<hr>

Save the changes by pressing ```Ctrl+s```.

## Step 5.2: Implementing the parser

Click on [parser.p4](./labs_files/lab3/parser.p4) to open the code in the editor.

<img src="./labs_files/lab3/figs/5_02_01.png" width="700px"><br>

We can see that the headers.p4 file that we just filled out is included here in the parser. The file also includes a starter code that declares a parser named MyParser. Note how the headers and the metadata structs that we defined previously are passed as parameters to the parser. 

<hr>

Add the start state inside the parser by inserting the following code.

    state start {
         transition parse_ethernet;
    }

<img src="./labs_files/lab3/figs/5_02_02.png" width="700"><br>

The start ```state``` is the state where the parser begins parsing the packet. Here we are transitioning unconditionally to the ```parse_ethernet``` state.

<hr>

Add the parse_ethernet state inside the parser by inserting the following code.

    state parse_ethernet {
       packet.extract(hdr.ethernet);
       transition select(hdr.ethernet.etherType) {
       	TYPE_IPV4: parse_ipv4;
       	default: accept;
       }
    }

<img src="./labs_files/lab3/figs/5_02_03.png" width="700"><br>

The ```parse_ethernet``` state extracts the Ethernet header and checks for the value of the header field ```etherType```. Note how we reference a header field by specifying the header to which that field belongs (i.e., ```hdr.ethernet.etherType```). If the value of ```etherType``` is ```TYPE_IPV4``` (which corresponds to 0x0800 as defined previously), the parser transitions to the ```parse_ipv4 state```. Otherwise, the execution of the parser terminates.

<hr>

Add the parse_ipv4 state inside the parser by inserting the following code.

    state parse_ipv4 {
       packet.extract(hdr.ipv4);
       transition select(hdr.ipv4.protocol) {
       	TYPE_TCP: parse_tcp;
       	default: accept;
       }
    }

<img src="./labs_files/lab3/figs/5_02_04.png" width="700"><br>

The``` parse_ipv4``` state extracts the IPv4 header and checks for the value of the header field ```protocol```. Note how we reference a header field by specifying the header to which that field belongs (i.e., ```hdr.ipv4.protocol```). If the value of ```protocol``` is ```TYPE_TCP``` (which corresponds to 6 as defined previously), the parser transitions to the ```parse_tcp``` state. Otherwise, the execution of the parser terminates.

<hr>

Add the parse_tcp state inside the parser by inserting the following code.

    state parse_tcp {
        packet.extract(hdr.tcp);
        transition accept;
    }

<img src="./labs_files/lab3/figs/5_02_05.png" width="700"><br>

The ```parse_tcp``` state extracts the TCP header and terminates the execution of the parser.

<hr>

Save the changes by pressing ```Ctrl+s```.

## Step 5.3: Compiling the P4 code
To upload all needed P4 codes and compile main.p4, issue the following command.

In [63]:
server1.upload_file('labs_files/lab3/headers.p4','headers.p4')
server1.upload_file('labs_files/lab3/parser.p4','parser.p4')
server1.upload_file('labs_files/lab3/precontrol.p4','precontrol.p4')
server1.upload_file('labs_files/lab3/control.p4','control.p4')
server1.upload_file('labs_files/lab3/deparser.p4','deparser.p4')
server1.upload_file('labs_files/lab3/main.p4','main.p4')
stdout, stderr = server1.execute(f'sudo p4c-dpdk --arch=pna main.p4 -o lab3.spec')
stdout, stderr = server1.execute(f'ls')

control.p4
deparser.p4
dpdk
ethdev.io
headers.p4
host_tune.sh
lab3.cli
lab3.spec
main.p4
p4c
parser.p4
precontrol.p4
run_pipeline.sh
sender.py
set_topology.sh
tmpfile


The command above invokes the ```p4c-dpdk``` compiler to compile the ```main.p4``` program and generates the ```lab3.spec``` file which is a specification file needed to run the pipeline.

# Step 6: Running the P4-DPDK pipeline and the lab topology
This section shows the steps required to run the P4-DPDK along with building the lab topology. In this lab, the procedure is automated.

## Step 6.1: Uploading required scripts
The code below uploads to server1 the CLI and I/O scripts, along with other scripts needed to automate the process of running the pipeline and building the topology.

In [55]:
server1.upload_file('labs_files/lab3/lab3.cli','lab3.cli')
server1.upload_file('labs_files/lab3/ethdev.io','ethdev.io')
server1.upload_file('labs_files/lab3/run_pipeline.sh','run_pipeline.sh')
server1.upload_file('labs_files/lab3/set_topology.sh','set_topology.sh')
server1.upload_file('labs_files/lab3/sender.py','sender.py')
stdout, stderr = server1.execute(f'chmod +x run_pipeline.sh')
stdout, stderr = server1.execute(f'chmod +x set_topology.sh')
stdout, stderr = server1.execute(f'chmod +x sender.py')

## Step 6.2: Opening a terminal
Launch a new terminal by opening a new tab and then select "terminal".

<img src="./labs_files/lab3/figs/6_02_terminal.gif" width="1000px"><br>

Copy the output of the command below and paste it into the terminal to enter server1.

In [14]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@132.249.252.178'

## Step 6.3: Running the P4-DPDK pipeline
Build and run the pipeline by typing the following command:

    sudo ./run_pipeline.sh
    
<img src="./labs_files/lab3/figs/6_03.png" width="650px"><br>

The ```run_pipeline.sh``` script is a shell script that automates the process of running the P4-DPDK pipeline. In this lab, part of the pipeline output is hidden to display only the relevant logs.

## Step 6.4: Building the lab topology
The code below uploads to server1 the CLI and I/O scripts, along with other scripts needed to automate the process of running the pipeline and building the topology.

In [69]:
stdout, stderr = server1.execute(f'sudo ./set_topology.sh')

The set_topology.sh script is a shell script that automates the process of building the lab topology. Two namespaces are built and configured in this step with a virtual device linked to each as shown in the figure below.

<img src="./labs_files/lab1/figs/6_04_ip.png" width="550px"><br>

# Step 7: Testing the application

### Step 7.1: Sending an ethernet packet from h1 to h2
To test the application, we will send a packet with an ethernet header from h1 to h2 by running the provided Python script sender.py 

In [70]:
stdout, stderr = server1.execute(f'sudo ip netns exec h1 python3 sender.py -r h2 -ph eth')

. WARNING: Interface lo: no address assigned
 Receiving host: h2
Packet headers: eth

Sent 1 packets.


Running the Python script requires two parameters:

•	```-r```: receiving host (h1 or h2) <br>
•	-```ph```: included packet headers (eth or eth/ipv4 or eth/ipv4/tcp)


### Step 7.2: Inspect the pipeline terminal

Observe the DPDK logs at the bottom of the terminal. These logs correspond to the packet processing function executed in the .spec file generated when the P4 code is compiled. 

<img src="./labs_files/lab3/figs/7_02.png" width="800px"><br>

The “extract header” log followed by the header ID “0” in the grey box, corresponds to the parsed ethernet header with the size of 14 bytes (112 bits) as declared in the headers.p4 file. The header ID is an indicator of the parsed header order. Therefore “header 0” is the first parsed header. Note that the IPv4 header (header 1) and the TCP header (header 2) were not extracted and not emitted (invalid) since they were not included in the sent packet.

The “tx 1 pkt to port 1” log indicated that one packet has been sent to port 1 which corresponds to host 2.

For a more readable output press enter in the terminal a few times (five times) to provide space for the next logs.

### Step 7.3: Sending an IP packet from h1 to h2
We will send a packet with an Ethernet and IPv4 header from h1 to h2 by running the provided Python script sender.py.

In [71]:
stdout, stderr = server1.execute(f'sudo ip netns exec h1 python3 sender.py -r h2 -ph eth/ipv4')

. WARNING: Interface lo: no address assigned
 Receiving host: h2
Packet headers: eth/ipv4

Sent 1 packets.


### Step 7.4: Inspect the pipeline terminal

<img src="./labs_files/lab3/figs/7_04.png" width="800px"><br>

The “extract header” log appears twice in the grey box, corresponding to the parsed ethernet header (header 0) with a size of 14 bytes (112 bits) and the parsed IPv4 header (header 1) with a size of 20 bytes (160 bits) as declared in the headers.p4 file. Note that the TCP header (header 2) was not extracted and not emitted (invalid) since it was not included in the sent packet.

The “tx 1 pkt to port 1” log indicated that one packet has been sent to port 1 which corresponds to host 2.

For a more readable output press enter in the terminal a few times (five times) to provide space for the next logs.

### Step 7.5: Sending an TCP packet from h1 to h2
We will send a packet with an ethernet, IPv4, and TCP header from h1 to h2 by running the provided Python script sender.py.

In [72]:
stdout, stderr = server1.execute(f'sudo ip netns exec h1 python3 sender.py -r h2 -ph eth/ipv4/tcp')

. WARNING: Interface lo: no address assigned
 Receiving host: h2
Packet headers: eth/ipv4/tcp

Sent 1 packets.


### Step 7.6: Inspect the pipeline terminal

<img src="./labs_files/lab3/figs/7_06.png" width="800px"><br>

The “extract header” log appears three times in the grey box, corresponds to the parsed ethernet header (header 0) with size of 14 bytes (112 bits), the parsed IPv4 header (header 1) with size 20 bytes (160 bits) and the parsed TCP header (header 2) with size 20 bytes (160 bits) as declared in the headers.p4 file. The “tx 1 pkt to port 1” log indicated that one packet has been sent to port 1 which corresponds to host 2.

For a more readable output press enter in the terminal a few times (five times) to provide space for the next logs.

### Step 7.7: Stopping the DPDK pipeline
Stop the DPDK pipeline by pressing ```ctrl+c``` in the terminal running the pipeline.

<img src="./labs_files/lab3/figs/7_07.png" width="800px"><br>

Note that the logs in the terminal correspond to the code executed for packet processing. 

# Step 8: Delete the slice

This concludes Lab 3. Please delete your slice when you are done with your experiment.

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice = fablib.get_slice(name="P4DPDK_lab3_vnic")
slice.delete()

# References

1.	The P4 Language Consortium, “P4 Portable NIC Architecture (PNA)”, Version 0.5, 2021. [Online]. Available: https://p4.org/p4-spec/docs/PNA.html
2.	P4lang, “p4c core.p4”. [Online]. Available: https://github.com/p4lang/p4c/blob/main/p4include/core.p4.
3.	P4lang, “pna.p4”, [Online]. Available: https://github.com/p4lang/pna/blob/main/pna.p4
4.	P4 Language Tutorial. [Online]. Available: https://tinyurl.com/2p9cen9e.